STA 220 Homework 2
===========

- Do not distribute
- Do the entire homework here in the notebook by adding cells below the given exercise.
- When turning the homework in simply submit your notebook file to canvas.
- Obviously, do not copy the code from some online source or the other students.
- After finishing the homwork, please select Kernel -> Restart & Run All, then save, submit

## Your Name: Chenghan Sun

## Your Student ID: 915030521

The multi-armed bandit framework assumes that you are in a two player game where you select a number from 1 to K (we call this number the arm that you pulled).  Then the other player selects a reward, $r_{i,t}$ based on the arm, $i$, at time $t$ that you selected and reveals that to you.  Your job is to come up with a policy which determines which arm to pull at a given time based on the past performances of the arms.

The name multi-armed bandit comes from the gambling world, in which a slot machine is called a one armed bandit.  In that setting, you pull the arm and recieve some reward.  In this fictional setting there are multiple arms for the slot machine, each paying out different rewards.  Because you can only pull one at a time, you only see the reward from the arm you pulled.  This partial observability puts you in the challenging position of needing to explore the arms, seeing which has better performance, before you start to exploit the best arm.  Below is a simulation from a simple mult-armed bandit.

In [1]:
import numpy as np
import time
from matplotlib import pyplot as plt
plt.style.use('ggplot')

In [37]:
class SimpleBandit:
    '''
    The bandit class you will use in this homework. DO NOT modify
    '''
    def __init__(self):
        self._mu = np.array([-1.,-2.,1.5,0.5,-0.25,.75,.1,1.8,-3])
        self._p = 1 / (1 + np.exp(-self._mu))
        self.num_arms = len(self._mu)
        self.total_rewards = np.zeros(len(self._mu))
        
    def pull(self,arms):
        self.current_rewards = np.random.binomial(1,self._p)
        self.total_rewards += self.current_rewards
        return self.current_rewards[arms]

In [38]:
np.random.seed(5)
band = SimpleBandit()  # reset
[band.pull(2) for t in range(10)]  # do arm 1 for 10 times 
# band.pull(2)  # set one arm and only pull once 

[1, 1, 1, 1, 1, 1, 0, 1, 1, 1]

In the above we see that if we had pulled the 1 arm 10 times then our total reward for that arm is 2 because it returned a 1 twice.  Our rewards are binary, only 0 or 1.  You can also see the total rewards from each arm below.

In [39]:
band.total_rewards  # for each arm 

array([2., 2., 9., 6., 2., 6., 3., 8., 0.])

In [40]:
print(band.current_rewards)  # current rewards list for each arm 
print(band.total_rewards)  # total rewards for each arm w.r.t 10(above) time steps

[0 0 1 1 0 1 0 1 0]
[2. 2. 9. 6. 2. 6. 3. 8. 0.]


One simple policy is to always pull the arm 2, and another is to pull the arm 1.  We can compare these with the following:

In [41]:
rewards = np.array([band.pull([1,2]) for t in range(100)])

rewards now has the rewards for each policy in each column, we can compare the rewards for these policies below.

In [42]:
rewards.sum(axis=0)  # first two arms, 100 time steps 

array([14, 90])

It seems that 'always pull 2' is a better policy.  Another simple policy is to randomly select an arm and pull it.  This can be seen as a pure exploration policy.  All of your policies should have the select_arm method which tells you which arm to pull, and the update_reward method which updates any internal state information based on the observed reward.  In this case nothing needs to be updated.

In [43]:
class RandomPolicy:
    """
    Random policy, pure exploration. DO NOT modify
    """
    def __init__(self, num_arms):
        self.num_arms = num_arms
        self.current_arm = None
        
    def select_arm(self):
        """
        choose which arm to pull
        """
        self.current_arm = np.random.randint(self.num_arms)
        return self.current_arm
    
    def update_reward(self, reward):
        """
        enter observed reward
        """
        return None

**Exercise 1.** The regret of a bandit policy is the difference between the total reward you would get from the "best" arm in hindsight and the total reward your policy recieved.  So if your policy selected arms $i_1,\ldots,i_T$ for total of $T$ time, then the regret is 
$$\max_j \sum_{t=1}^T r_{j,t} - \sum_{t=1}^T r_{i_t,t}.$$
The SimpleBandit class maintains what reward you would have recieved if you just pulled a given arm in the ``total_rewards`` attribute (`current_rewards[i]` is the total reward recieved if i is pulled each time).  Fill the def below which takes a list of policies to play over T time steps and returns a list of regrets for each policy.  Test it on the simple bandit and random policy.

In [44]:
# For t in len(T):
    # use rand(9)
    # do multiple policy choice on rand(9), which gives list of n arms randomly
    # input the list of arms to the bandit(), which returned the total rewards and current rewards
    # do the same subtraction to find regrets

In [45]:
def run_trajectory(bandit, policies, T):
    """
    Run T steps of bandit pulling each policy in each time step
    
    Arguments: 
    bandit: 
        a fresh instance of a Bandit class, 
        in this homework you will be always using an instance from RandomPolicy class
    
    policies:
        a list like [policy1, policy2, policy3 ...]
        each of the policy will have select_arm method and update_reward method
    
    Output: 
        regret of each policy in list, like
        [regret1, regret2, ...]
    """
    arms_cumulative_rewards = np.zeros(len(policies))  # initialize the policy space 
    for t in range(T):  # iterate all time steps
        selected_arms = [p.select_arm() for p in policies]  # select random arm T times under each policy
        # print(f"Time step {t} selects these arms: {selected_arms} under each policy")
        arms_current_rewards = bandit.pull(selected_arms)  # pull, pass to policy class
        
        global CurRewardList_EGP
        global CurRewardList_UCB
        if 'Epsilon_Greedy_Policy' in str(policies):
            CurRewardList_EGP.append(arms_current_rewards[1])  # append to global scope 
        elif 'UCB' in str(policies):
            CurRewardList_UCB.append(arms_current_rewards[1])  # append to global scope 
        arms_cumulative_rewards += arms_current_rewards
        # print(f"The current binary rewards for the {len(selected_arms)} arms: {arms_current_rewards}")
        # print(f"The cumulative rewards at {t} time step are: {arms_cumulative_rewards}")
        # print(f"The potential total rewards for each possible arm: {bandit.total_rewards}")
        regrets_history = [np.amax(bandit.total_rewards) - cu_reward for cu_reward in arms_cumulative_rewards]
        # print(f"The regrets history at the {t}th time step for the {len(policies)} arms: \033[91m{regrets_history}\033[0m")
    return regrets_history

In [46]:
# test code, do not change
t1 = time.time()
num_arms = SimpleBandit().num_arms
p = RandomPolicy(num_arms)
policies = [p]
regrets = [run_trajectory(SimpleBandit(), policies, 1000) for _ in range(10)]
print('Time Usage : {}'.format(time.time() - t1))
print(regrets)
np.mean(regrets)

Time Usage : 0.45552587509155273
[[375.0], [345.0], [350.0], [353.0], [397.0], [347.0], [366.0], [366.0], [383.0], [385.0]]


366.7

**Exercise 2.** epsilon-greedy is a policy that has a few roughly equivalent variants, but the one we will use is the following:  For each time step, with probability epsilon, pull an arm at random, otherwise pull the current best arm.  The current best arm is the one which has the most **mean** reward up to that time.  If there is a tie for current best policy, break the tie randomly.

Implement epsilon-greedy in the following class, test it by having it compete once with the random policy for epsilon=0.1 for T = 1000 time points.

In [48]:
class Epsilon_Greedy_Policy:
    """ 
    Epsilon-greedy algorithm:
        With probability epsilon, pull an arm at random, otherwise pull the current best arm.
        The current best arm is the one which has the most total reward up to that time
    """
    def __init__(self, num_arms, eps):
        self.num_arms = num_arms  # total number of arms 
        self.current_arm = None
        self.eps = eps  # probability epsilon for decision making 
        self.t = 0  # initialized time step 
        self.cumulative_rewards = np.zeros(self.num_arms)
        self.arms_counter = np.zeros(self.num_arms)
        self.arms_performance = np.zeros(self.num_arms)  # update arms mean rewards dynamically 
        
    def greedy_mode(self):
        """ Greedily policy: 
                decision made dynamically with updated performance of bandit for each pull 
        """
        self.arms_counter, self.cumulative_rewards = self.update_reward()
        # update arms performance history 
        for arm in range(self.num_arms):
            if self.arms_counter[arm] != 0:
                self.arms_performance[arm] = self.cumulative_rewards[arm] / self.arms_counter[arm]
        # print(f"The arms_performance: {self.arms_performance}")
        
        # pure exploit
        greedy_choice = np.argmax(self.arms_performance)  # the arm with the most mean reward
        # if there is a tie for current best policy
        # collect all the arm indices with 
        ties = [index for index, value in enumerate(self.arms_performance) if value == greedy_choice]
        if len(ties) > 1:
            greedy_choice = np.random.choice(ties)  # break the tie randomly
        return greedy_choice
    
    def select_arm(self):
        """ Choose arms based on Epsilon-greedy:
            Apply:
                Random Policy -- explore -- RandomPolicy()
            or
                greedy policy -- exploit -- self.greedy_mode()
        """
        rand_flag = np.random.rand()
        # print(f"The random number for this time: {rand_flag}")
        if rand_flag < self.eps:
            self.arms_counter, self.cumulative_rewards = self.update_reward()  # update reward information
            self.current_arm = RandomPolicy(self.num_arms).select_arm()  # apply explore
        elif rand_flag > self.eps:
            self.current_arm = self.greedy_mode()  # apply exploit
        else:  # in case rand_flag = self.eps
            policies = [RandomPolicy(self.num_arms).select_arm(), self.greedy_mode()]
            choice_index = np.random.choice([0,1])
            self.current_arm = policies[choice_index]
        self.t += 1
        return self.current_arm
        
    def update_reward(self):
        """ 
        """
        # print(f"The selected arm from last Epsilon-greedy: {self.current_arm}")
        if self.current_arm == None:
            return self.arms_counter, self.cumulative_rewards
        
        global CurRewardList_EGP
        current_reward = CurRewardList_EGP[self.t-1]  # receive current reward from global scope 
        # print(f"The reward of last pull: {current_reward}")
        self.arms_counter[self.current_arm] += 1  # update known-arms space
        self.cumulative_rewards[self.current_arm] += current_reward  # update rewards histroy
        # print(f"The currect cumulative rewards: {self.cumulative_rewards}")  # initialized with 0s
        # print(f"The currect count of arm space: {self.arms_counter}")
        return self.arms_counter, self.cumulative_rewards


In [53]:
""" DEMO
"""
t1 = time.time()
CurRewardList_EGP = []
num_arms = SimpleBandit().num_arms
rp = RandomPolicy(num_arms)
eg = Epsilon_Greedy_Policy(num_arms, 0.1)
policies = [rp, eg]
regrets = [run_trajectory(SimpleBandit(), policies, 1000) for _ in range(10)]
print(regrets)
print('Time Usage : {}'.format(time.time() - t1))
np.mean(regrets)

[[360.0, 53.0], [355.0, 32.0], [364.0, 29.0], [418.0, 42.0], [369.0, 28.0], [372.0, 31.0], [371.0, 31.0], [367.0, 29.0], [360.0, 39.0], [375.0, 44.0]]
Time Usage : 0.6622772216796875


203.45

In [14]:
len(CurRewardList_EGP)

10000

In [15]:
# test code
t1 = time.time()
regrets = []

# Your code here, 
    # generate 10 regrets with simple bandit and epsilon-greedy policy, 
    # save results in regrets,
    # refer to second block in exercise 1

print('Time Usage : {}'.format(time.time() - t1))
np.mean(regrets)

Time Usage : 7.867813110351562e-05


/Users/furinkazan/Library/Python/3.7/lib/python/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/furinkazan/Library/Python/3.7/lib/python/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


nan

**Exercise 3.** Exp3 is a policy that is more nuanced.  The idea is that at each time point you pull an arm with some probability `pi[i]` which is updated based on the performance of the arm.  Look at the full algorithm in http://proceedings.mlr.press/v24/seldin12a/seldin12a.pdf, Algorithm 1.  

Implement this version of the Exp3 algorithm in the following class, test it by having it compete once with the random policy for T = 1000 time points.

In [57]:
class Exp3:
    """ 
    Exp3 algorithm:
        
    """
    def __init__(self, num_arms):
        self.num_arms = num_arms
        self.current_arm = None
        self.t = 0  # initialized time step 
        self.weights = np.ones(self.num_arms)  # initialize weights to be 1
        self.prob_dist = np.zeros(self.num_arms)  # initialize probability distributions to be 0
        
        
    def choose_rate(self):
        """  
        Choose the learning rate εt:
            which is equal to the exploration rate, is changing with time
        """
        epsi_t = min(1. / self.num_arms, np.sqrt(np.log(self.num_arms) / (self.t * self.num_arms)))
        return epsi_t
    
    def draw(self,):
        """ set distribution 
        """
        
        
    def select_arm(self):
        """
        """
        total_weights = sum(self.weights)
        epsi_t = self.choose_rate()
        for arm in range(self.num_arms):
            # update probability distributions
            self.prob_dist[arm] = (1 - epsi_t) * (self.weights[arm] / total_weights) + epsi_t / self.num_arms
        
        self.t += 1
        return 
    
    def update_reward(self, reward):
        None
        


In [58]:
def distr(weights, gamma=0.0):
    theSum = float(sum(weights))
    return tuple((1.0 - gamma) * (w / theSum) + (gamma / len(weights)) for w in weights)

array([1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [18]:
# test code
t1 = time.time()
regrets = []

print('Time Usage : {}'.format(time.time() - t1))
np.mean(regrets)

Time Usage : 7.295608520507812e-05


nan

**Exercise 4.** The Upper Confidence Bound (UCB) algorithm is intuitive.  You pull the arm that has the largest upper confidence bound for the mean reward.  So if you have an upper bound on the mean reward for an arm this is either because the arm is performing well or you have not pulled the arm much, resulting in a wide confidence interval.  The UCB for arm i is then,
$$ U_{i,t} = \hat \mu_{i,t} + C \sqrt{\frac{\log(t)}{n_{i,t} + 1}}$$
where $\hat \mu_{i,t}$ is the mean reward of the arm up to time $t$, $n_{i,t}$ is the number of times that that arm has been pulled.  $C$ is an argument (can be taken to be 2 by default).

Implement this version of the UCB algorithm in the following class, test it by having it compete once with the random policy for T = 1000 time points with $C = 2$.

In [19]:
class UCB:
    """
    Upper Confidence Bound (UCB) algorithm:
        pull the arm that has the largest upper confidence bound for the mean reward
    """
    def __init__(self, num_arms, C):
        self.num_arms = num_arms  # total number of arms 
        self.current_arm = None
        self.C = C
        self.t = 0  # initialized time step 
        self.arms_counter = np.zeros(self.num_arms)
        self.arms_performance = np.zeros(self.num_arms)  # update arms mean rewards dynamically 
        
    def select_arm(self):
        """ 
        Apply UCB decision rule 
        """
        self.arms_counter, self.arms_performance = self.update_reward()
        if self.current_arm == None:
            self.current_arm = RandomPolicy(self.num_arms).select_arm()  # tie for the first pull 
            return self.current_arm
        pull_sum = sum(self.arms_counter)
        u_space = [self.arms_performance[a] + 
                   np.sqrt((self.C * np.log(pull_sum)) / (self.arms_counter[a] + 1)) 
                   for a in range(self.num_arms)]
        self.current_arm = np.argmax(u_space)
        
        # if there is a tie for current best policy
        ties = [index for index, value in enumerate(self.arms_performance) if value == self.current_arm]
        if len(ties) > 1:
            self.current_arm = np.random.choice(ties)  # break the tie randomly
        self.t += 1
        return self.current_arm
        
    def update_reward(self):
        """ 
        Update:
            1. arm space
            2. arm mean performance 
        """
        if self.current_arm == None:
            return self.arms_counter, self.arms_performance  # exit for the first pull 
        global CurRewardList_UCB
        current_reward = CurRewardList_UCB[self.t-1]  # receive current reward from global scope
        self.arms_counter[self.current_arm] += 1  # update known-arms space
        # update rewards histroy
        self.arms_performance[self.current_arm] += (
            current_reward - self.arms_performance[self.current_arm]) / self.arms_counter[self.current_arm]   
        return self.arms_counter, self.arms_performance
    

In [20]:
""" DEMO Ex 4
"""
t1 = time.time()
CurRewardList_UCB= []
num_arms = SimpleBandit().num_arms
rp = RandomPolicy(num_arms)
ucb = UCB(num_arms, 2)
policies = [rp, ucb]
regrets = [run_trajectory(SimpleBandit(), policies, 1000)  for _ in range(10)]
print(regrets)
print('Time Usage : {}'.format(time.time() - t1))
np.mean(regrets)

[[355.0, 190.0], [389.0, 71.0], [372.0, 45.0], [374.0, 47.0], [358.0, 28.0], [421.0, 19.0], [422.0, 22.0], [358.0, 21.0], [396.0, 18.0], [352.0, 14.0]]
Time Usage : 1.1155729293823242


213.6

In [21]:
print(len(CurRewardList_UCB))

10000


In [22]:
# test code
t1 = time.time()
regrets = []


print('Time Usage : {}'.format(time.time() - t1))
np.mean(regrets)

Time Usage : 4.673004150390625e-05


nan

**Exercise 5.** 

1. Modify the `run_trajectory` def to output all of the regrets up to that time in a T x K (for K policies) array.
2. Try 4 different values of $\epsilon$ for epsilon-greedy: 0.01, 0.05, 0.1, 0.2 and have them compete.  Plot the regrets as a function of t.  Note the best performing selection of $\epsilon$ at $T = 1000$.
3. Try 4 different values of $C$ in UCB: 0.5, 0.75, 1, 2 and have them compete.  Plot the regrets as a function of t.  Note the best performing selection of $C$ at $T = 1000$.
4. Using the optimal values of $C$ and $\epsilon$ have all four methods compete. Plot the regrets as a function of t, remark on if the relative performance changes over time.  Is one algorithm always dominant?  Make any other conclusions.

**Note**: 
  + Please finishe 4 parts in 4 code blocks.
  + For part 2, 3, and 4 you should call `run_trajectory` in each of them. In the end of each part, you need to write a markdown block to explain your findings and state your conclusion.
  + For each part, set figure size to (17, 8), plot all lines in the same graph and set proper title. DO NOT create more than 1 subplots. Include legend in your plots so that they are easy to read.

In [23]:
# Part 1. Write run_trajectory function here again, do not modify the above block

In [24]:
# Part 2 code, output your plot

### Findings and conclusion for part 2

In [25]:
# Part 3 code, output your plot

### Findings and conclusion for part 3

In [26]:
# Part 4 code, output your plot

### Findings and conclusion for part 4